Subset 1 (s1): origin US, destination US corr to Anal_0 <br>
Subset 2 (s2): origin AFW, destination all corr to Anal_7 <br>
Subset 3 (s3): origin all, destination AFW corr to Anal_8 <br>
Subset 4 (s4): origin KIS, destination all corr to Anal_9<br>
Subset 5 (s5): origin all, destination KIS corr to Anal_10<br>


Analysis 0: origin US, dest US <br>
Analysis 1: origin US, dest non-US <br>
Analysis 2: origin non-US, dest US <br>
Analysis 3: origin all, dest all<br>
Analysis 4: origin all states, dest all<br>
Analysis 5: origin all, dest all states<br>
Analysis 6: origin US, dest all<br>
Analysis 7: origin AFW, destination all <br>
Analysis 8: origin all, destination AFW<br>
Analysis 9: origin KIS, destination all<br>
Analysis 10: origin all, destination KIS<br>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import sys, os, pathlib, shutil, platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import lilliefors

from prophet import Prophet

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import plot_cross_validation_metric

# import plotly.graph_objs as go
import plotly.express as px

import math

In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
 
plt.rcParams['figure.figsize']=(20,10)

In [ ]:
def load_datsets(data_path, years_list, analysis_type_list):
    """
    Concatanates a list of datasets into one dataframe

    Returns:
    Concatanated list as a dataframe
    """
    full_path_list = [data_path+str(crt_year) + '_analysis'+ str(analysis_type) +'_' + str(regional_flag)+'_'+str(aircraft_flag) + '.csv' 
                      for crt_year in years_list 
                      for analysis_type in analysis_type_list]
    print(full_path_list[0])
    allDatasets= pd.concat([pd.read_csv(str(crt_file_name)) for crt_file_name in full_path_list], keys=years_list).reset_index()
    return allDatasets

In [ ]:
data_path='./../../../data/paav_cargo/agg_data/freight_aggregation__' 

analysis_type_list = [analysis_type for analysis_type in range(11)]
regional_flag = "regional"
# regional_flag options: 'alldistance', 'regional'
aircraft_flag = "jets"
# aircraft_flag options:'jetsandnonjets', 'nonjets', 'jets'

#use these for s2,s3,s4,s5
years_list=[yr for yr in range (1990,2024)]
years_list_noCov=[yr for yr in range (1990,2020)]

#use these for s1
years_list=[yr for yr in range (2004,2024)]
years_list_noCov=[yr for yr in range (2004,2020)]

analysis_type_list = [0]
preprocessed_data_s1 = load_datsets(data_path, years_list, analysis_type_list)

analysis_type_list = [7]
preprocessed_data_s2 = load_datsets(data_path, years_list, analysis_type_list)

analysis_type_list = [8]
preprocessed_data_s3 = load_datsets(data_path, years_list, analysis_type_list)

analysis_type_list = [9]
preprocessed_data_s4_noCov = load_datsets(data_path, years_list_noCov, analysis_type_list)

analysis_type_list = [9]
preprocessed_data_s4 = load_datsets(data_path, years_list, analysis_type_list)

analysis_type_list = [10]
preprocessed_data_s5 = load_datsets(data_path, years_list, analysis_type_list)

In [ ]:
# years_list = [yr for yr in range (1990,2024)]

# these are the years lists we use when regional_flag = regional and aircraft_flag = nonjets
# years_list_s1 = [yr for yr in range (2004,2024)]
# years_list_s5 = [yr for yr in range (2002,2024)]
# years_list_s2 = [yr for yr in range (2002,2020)]
# years_list_s3 = [yr for yr in range (2002,2020)]
# years_list_s4 = [yr for yr in range (2002,2020)]

# these are the years lists we use when regional_flag = regional and aircraft_flag = jetsandnonjets
# years_list_s1 = [yr for yr in range (2004,2024)]
# years_list_s5 = [yr for yr in range (2002,2024)] #starts in oct2002
# years_list_s2 = [yr for yr in range (2002,2024)] #basically starts in oct2002
# years_list_s3 = [yr for yr in range (2002,2024)]
# years_list_s4 = [yr for yr in range (2002,2020)] # real one
years_list_s4 = [yr for yr in range (2002,2024)]
years_list_s5_noCov = [yr for yr in range (2002,2020)]


# these are the years lists we use when regional_flag = regional and aircraft_flag = jets
# years_list_s1 = [yr for yr in range (2004,2024)]
# # years_list_s5 = [yr for yr in range (1990,2024)] #interesting, buncha zeroes
# years_list_s2 = [yr for yr in range (2002,2024)] 
# years_list_s3 = [yr for yr in range (2002,2024)]
years_list_s4 = [yr for yr in range (1990,2024)] #interesting, buncha zeroes
years_list_s4_noCov = [yr for yr in range (1990,2020)]

In [ ]:
preprocessed_data_s4.head(20)

In [ ]:
preprocessed_data_s4_noCov.head(20)

In [ ]:
preprocessed_data_s5.tail(3)

In [ ]:
preprocessed_data_s5.head(20)

In [ ]:
preprocessed_data_s1_noCov.head(3)
preprocessed_data_s1_noCov.head(20)
preprocessed_data_s1_noCov.tail(3)

In [ ]:
## below code makes yearly aggregations
# preprocessed_data.drop(columns=['level_1', 'level_0'],axis=1,inplace=True)
# # preprocessed_data = preprocessed_data.rename(columns = {'level_0':'year'})
# # my_testing["year"] = my_testing['ds'].dt.year #tagged this cuz level_0 column takes care of it
# preprocessed_data = preprocessed_data.groupby(['year', 'origin', 'dest'], as_index=True, group_keys=True)['y', 'num flights' ].agg(['sum','count'])
# preprocessed_data.reset_index(inplace=True)
# preprocessed_data.columns= ['_'.join(col) for col in preprocessed_data.columns.values]
# preprocessed_data.drop(['origin_', 'dest_', 'y_count', 'num flights_count','num flights_sum'], axis=1, inplace = True)
# preprocessed_data.rename(columns={'y_sum': 'y', 'year_':'ds' }, inplace = True)
# preprocessed_data.head(3)

In [ ]:
forplot_s4 = preprocessed_data_s4.copy(deep=True)
forplot_s4 = preprocessed_data_s4[['ds', 'y']]
forplot_s4.index = pd.to_datetime(forplot_s4['ds'],format='%Y-%m-%d')
forplot_s4.drop(columns=['ds'],inplace=True)
forplot_s4.sort_index(inplace=True)
forplot_s4
forplot_s4.plot()

In [ ]:
forplot_s1_noCov = preprocessed_data_s1_noCov.copy(deep=True)
forplot_s1_noCov = preprocessed_data_s1_noCov[['ds', 'y']]
forplot_s1_noCov.index = pd.to_datetime(forplot_s1_noCov['ds'],format='%Y-%m-%d')
forplot_s1_noCov.drop(columns=['ds'],inplace=True)
forplot_s1_noCov.sort_index(inplace=True)
forplot_s1_noCov
forplot_s1_noCov.plot()

In [ ]:
decompose_data_s1 = seasonal_decompose(forplot_s1, model="additive", period=12)
decompose_data_s1.plot();

In [ ]:
decompose_data_s1_noCov = seasonal_decompose(forplot_s1_noCov, model="additive", period=12)
decompose_data_s1_noCov.plot();

In [ ]:
# plt.plot(decompose_data_s2.seasonal[:12])

In [ ]:
# plt.plot(decompose_data_s2.trend)

In [ ]:
def forecast_future(future_samples_count, df, growth = 'linear'):
    model = Prophet(growth=growth)
    model.fit(df)
    
    future = model.make_future_dataframe(periods=future_samples_count, freq='m')

    forecast = model.predict(future)
    return forecast, model

In [ ]:
forplot_s4=forplot_s4.reset_index()
forplot_s4

In [ ]:
forplot_s1_noCov=forplot_s1_noCov.reset_index()
forplot_s1_noCov

In [ ]:
future_samples_count= 120 # as of 2023, u will either input 168 months or 120 months for future_samples_count depending on ur last ds val printed above in order to forecast 10 yrs in the future
#if last ds val is in 2019, use 168 months to get to 2033. if last ds val is in 2023 then obviously use 120 months
forecasted_df_s4, fc_model_s4 = forecast_future(future_samples_count, forplot_s4)

In [ ]:
future_samples_count= 168 # as of 2023, u will either input 168 months or 120 months for future_samples_count depending on ur last ds val printed above in order to forecast 10 yrs in the future
#if last ds val is in 2019, use 168 months to get to 2033. if last ds val is in 2023 then obviously use 120 months
forecasted_df_s1_noCov, fc_model_s1_noCov = forecast_future(future_samples_count, forplot_s1_noCov)

In [ ]:
#this is not trend and seasonality; this is the forecast
plot_plotly(fc_model_s1_noCov, forecasted_df_s1_noCov)

In [ ]:
#this is not trend and seasonality; this is the forecast
plot_plotly(fc_model_s4, forecasted_df_s4)

In [ ]:
fig2 = fc_model_s4.plot_components(forecasted_df_s4,
                                 uncertainty = True,  plot_cap = True
                                 , weekly_start = 0,  yearly_start = 0)

fig2 = fc_model_s1_noCov.plot_components(forecasted_df_s1_noCov,
                                 uncertainty = True,  plot_cap = True
                                 , weekly_start = 0,  yearly_start = 0)

# Manual Decomposition of trend and seasonality
### We already have functions that do the breakdown above, this is just for demo purposes/ to explain what's being done

In [ ]:
reglist = forplot['y'].values.tolist()
reglist[:5]

seasonality_length = 12

def compute_trend(time_series_data_values, seasonality_length):
    trend_length = len(time_series_data_values)-seasonality_length+1
    trend = []
    for crtgroup in range(trend_length):
        crtavg = sum(time_series_data_values[:seasonality_length]) / seasonality_length
        trend.append(crtavg)
        time_series_data_values.pop(0)
    return trend
  
trend = compute_trend(reglist.copy(), seasonality_length)    
print(trend)
print(len(trend))
print(reglist)

In [ ]:
if (seasonality_length % 2) == 0: #even seasonality_length
    numdatestofillatstart = math.floor(seasonality_length/2) -1
    numdatestofillatend = math.floor(seasonality_length/2) 
else: #seasonality_length is odd
    numdatestofillatstart = math.floor((seasonality_length-1)/2) 
    numdatestofillatend = math.floor((seasonality_length-1)/2) 


In [ ]:
extendedtrend = trend.copy()
extendedtrend = [trend[0]] * numdatestofillatstart + extendedtrend
extendedtrend = extendedtrend+ [trend[-1]] * numdatestofillatend

print(trend)
print(extendedtrend)

In [ ]:
# plot trend
x_list = list(forplot.index.values)
plt.plot(x_list, extendedtrend, label='trend')
plt.legend()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
detrended = [element1 - element2 for (element1, element2) in zip(reglist, extendedtrend)]

print(detrended)
print(len(detrended))   

In [ ]:
# plot observed data, computed trend, and detrended data
plt.plot(x_list, reglist, label='observed')
plt.plot(x_list, extendedtrend, label='trend')
plt.plot(x_list, detrended, label='detrended')
plt.xticks(rotation = 45)
plt.legend()
plt.show()

In [ ]:
def grouptogether(yourlist, step):
    return [yourlist[index::step] for index in range(step)]
seasonalitygroupings = grouptogether(detrended, seasonality_length)
# print(seasonalitygroupings)

seasonality = [sum(crtmonth) / seasonality_length for crtmonth in seasonalitygroupings]
print(seasonality)
# print(len(seasonality))

In [ ]:
# plot seasonality (the seasonality_length=12 averages)
plt.plot(x_list[:seasonality_length], seasonality, label='seasonality')
plt.xticks(rotation = 45)
plt.legend()
plt.show()

In [ ]:
reconstructeddata = [extendedtrend[i] + seasonality[i%seasonality_length] for i in range(len(extendedtrend))]
print(reconstructeddata)

residuals = [reglist[i] - reconstructeddata[i] for i in range(len(reglist))]
print(residuals)

In [ ]:
# plot the residuals
plt.scatter(x_list, residuals, c ="red")
plt.xticks(rotation = 45)
plt.show()